In [57]:
import numpy as np
import sympy as smp
from scipy.integrate import solve_ivp
from numba import njit

In [58]:
t, k, m_s, m1, m2, l1, l2,g,k = smp.symbols("t k m_s m1 m2 l1 l2 g k")
x_s = smp.Function("x_s")
theta1 = smp.Function("theta1")
theta2 = smp.Function("theta2")

In [59]:
theta1 = theta1(t)
theta2 = theta2(t)
x_s = x_s(t)

In [60]:
theta1_d = smp.diff(theta1,t)
theta2_d = smp.diff(theta2,t)
x_s_d = smp.diff(x_s,t)

In [61]:
theta1_dd = smp.diff(theta1_d,t)
theta2_dd = smp.diff(theta2_d,t)
x_s_dd = smp.diff(x_s_d,t)

In [62]:
x1 = x_s + l1*smp.sin(theta1)
x2 = x1+l2*smp.sin(theta2)

y1 = -l1*smp.cos(theta1)
y2 = y1-l2*smp.cos(theta2)

In [63]:
K = smp.Rational(1,2)*((m1*(smp.diff(x1,t)**2+smp.diff(y1,t)**2))+(m2*(smp.diff(x2,t)**2+smp.diff(y2,t)**2))+(m_s*(smp.diff(x_s,t)**2))).simplify()
U = m1*g*y1+m2*g*y2+smp.Rational(1,2)*k*x_s**2
L = (K-U).simplify()

In [64]:
L

g*l1*m1*cos(theta1(t)) + g*m2*(l1*cos(theta1(t)) + l2*cos(theta2(t))) - k*x_s(t)**2/2 + m1*(l1**2*Derivative(theta1(t), t)**2 + 2*l1*cos(theta1(t))*Derivative(theta1(t), t)*Derivative(x_s(t), t) + Derivative(x_s(t), t)**2)/2 + m2*(l1**2*Derivative(theta1(t), t)**2 + 2*l1*l2*cos(theta1(t) - theta2(t))*Derivative(theta1(t), t)*Derivative(theta2(t), t) + 2*l1*cos(theta1(t))*Derivative(theta1(t), t)*Derivative(x_s(t), t) + l2**2*Derivative(theta2(t), t)**2 + 2*l2*cos(theta2(t))*Derivative(theta2(t), t)*Derivative(x_s(t), t) + Derivative(x_s(t), t)**2)/2 + m_s*Derivative(x_s(t), t)**2/2

In [65]:
E1 = smp.diff(L,theta1)-smp.diff((smp.diff(L,theta1_d)),t)
E2 = smp.diff(L,theta2)-smp.diff((smp.diff(L,theta2_d)),t)
E3 = smp.diff(L,x_s)-smp.diff((smp.diff(L,x_s_d)),t)

In [66]:
E3

-k*x_s(t) - m1*(-2*l1*sin(theta1(t))*Derivative(theta1(t), t)**2 + 2*l1*cos(theta1(t))*Derivative(theta1(t), (t, 2)) + 2*Derivative(x_s(t), (t, 2)))/2 - m2*(-2*l1*sin(theta1(t))*Derivative(theta1(t), t)**2 + 2*l1*cos(theta1(t))*Derivative(theta1(t), (t, 2)) - 2*l2*sin(theta2(t))*Derivative(theta2(t), t)**2 + 2*l2*cos(theta2(t))*Derivative(theta2(t), (t, 2)) + 2*Derivative(x_s(t), (t, 2)))/2 - m_s*Derivative(x_s(t), (t, 2))

In [67]:
sol = smp.solve([E1,E2,E3],(theta1_dd,theta2_dd,x_s_dd))

In [68]:
theta1_dd_f = smp.lambdify((theta1,theta2,x_s,theta1_d,theta2_d,x_s_d, m1, m2, m_s, k, l1, l2,g),sol[theta1_dd])
theta2_dd_f = smp.lambdify((theta1,theta2,x_s,theta1_d,theta2_d,x_s_d, m1, m2, m_s, k, l1, l2,g),sol[theta2_dd])
x_s_dd_f = smp.lambdify((theta1,theta2,x_s,theta1_d,theta2_d,x_s_d, m1, m2, m_s, k, l1, l2,g),sol[x_s_dd])

In [79]:
def dSdt(t,S,params):
    theta1,theta2,x_s,theta1_d,theta2_d,x_s_d = S
    m1,m2,m_s,k,l1,l2,g = params
    return [
        theta1_d,
        theta2_d,
        x_s_d,
        theta1_dd_f(theta1,theta2,x_s,theta1_d,theta2_d,x_s_d, m1, m2, m_s, k, l1, l2,g),
        theta2_dd_f(theta1,theta2,x_s,theta1_d,theta2_d,x_s_d, m1, m2, m_s, k, l1, l2,g),
        x_s_dd_f(theta1,theta2,x_s,theta1_d,theta2_d,x_s_d, m1, m2, m_s, k, l1, l2,g)
    ]

In [89]:
def rk4_step(f, t, y, dt, params):
    y = np.asarray(y, dtype=float)  # ensure NumPy array
    #print(f(t, y, params))
    k1 = np.asarray(f(t, y, params), dtype=float)
    k2 = np.asarray(f(t + dt/2, y + dt*k1/2, params), dtype=float)
    k3 = np.asarray(f(t + dt/2, y + dt*k2/2, params), dtype=float)
    k4 = np.asarray(f(t + dt, y + dt*k3, params), dtype=float)
    return y + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)
def integrate_system(f, y0, params, T=5.0, N=100):
    dt = T / N
    y = y0.copy()
    t = 0.0
    sol = np.zeros((N, len(y0)))
    for i in range(N):
        y = rk4_step(f, t, y, dt, params)
        sol[i]= y
        t += dt
    return sol

In [90]:
def get_solution_faster(system, y0, params, T=5.0,N=100):
    return integrate_system(system,y0,params)
    

In [91]:
params = (2,2,2,50,1,1,9.8)
get_solution_faster(dSdt,[0,0,3,0,0,0],params)

array([[ 9.20621673e-02,  6.50331440e-04,  2.90738861e+00,
         3.58647745e+00,  7.03223457e-02, -3.64061515e+00],
       [ 3.32498990e-01,  2.03145215e-02,  2.65327805e+00,
         5.64306567e+00,  9.09618278e-01, -6.26039517e+00],
       [ 6.17804323e-01,  1.09306914e-01,  2.30863757e+00,
         5.52565885e+00,  2.71722851e+00, -7.33827398e+00],
       [ 8.71063919e-01,  2.91161127e-01,  1.93093063e+00,
         4.52482093e+00,  4.50351296e+00, -7.69937915e+00],
       [ 1.06510190e+00,  5.52504809e-01,  1.54373263e+00,
         3.20824225e+00,  5.87153480e+00, -7.74501148e+00],
       [ 1.19121811e+00,  8.70175435e-01,  1.16099032e+00,
         1.85934937e+00,  6.75078621e+00, -7.51643326e+00],
       [ 1.25654806e+00,  1.21862479e+00,  7.97037805e-01,
         8.38549261e-01,  7.09509982e+00, -7.00201088e+00],
       [ 1.28538316e+00,  1.57081835e+00,  4.63138363e-01,
         4.13107428e-01,  6.92342528e+00, -6.35192796e+00],
       [ 1.30576781e+00,  1.90675191e+00,  1.606